In [39]:
import requests
import re
import datetime
import pickle
import random
import json
import time

In [64]:
# CONSTANTS
GECKO_COOKIE = '__cfduid=d0730edfccacd18b391f1ed3f9616683a1594354492; _vwo_uuid_v2=DF1D4D9E4431CC971F8A37D6C0B3F7E4A|df8cf9347bcc2c8a2d1dce7c43337b83; _ga=GA1.2.582244747.1594354493; _vwo_ssm=1; _vwo_uuid=DF1D4D9E4431CC971F8A37D6C0B3F7E4A; _ym_uid=1594354494795729336; _ym_d=1594354494; _vwo_ds=3%3Aa_0%2Ct_0%3A0%241594354492%3A89.41788852%3A%3A%3A54_0%2C7_0%2C6_0%2C5_0%2C2_0%3A0; cf_clearance=e4dcda26a6858e9a749e36af9a115b0ddb07cc12-1594592533-0-1zca1cc9eaz8b45487z4f9a6923-150; _vis_opt_s=2%7C; _vis_opt_test_cookie=1; _vwo_sn=238047%3A2; _gid=GA1.2.432605728.1594592541; _gat_gtag_UA_49392197_1=1; _session_id=29664855c746b51373e0f168674f6657; _ym_isad=1; _ym_visorc_57422968=b'
TARGET_COIN_SLUGS = [
    'BTC',    'ETH',    'XRP',    'BCH',    'BSV',    'LTC',    'BNB',
    'EOS',    'ADA',    'XTZ',    'LINK',    'XLM',    'XMR',    'NEO',
    'DASH',    'ZEC',    'BAT',    'ZRX',    'NANO',    'VET',    'COMP',
    'RLC'
]
start_date_code = "20130428"

# SEED RAND
random.seed(1)

# OTHER
weekly_dates_backup = []
print weekly_dates_backup

[]


In [41]:
def market_cap_crawler(weeks_to_crawl, pickled_weekly_dates):
    for datecode in weeks_to_crawl:
        print "crawling marketcap for "+datecode
        data_tuple = (datecode, get_total_market_cap(datecode))
        pickled_weekly_dates.append(data_tuple)
        pickle.dump(pickled_weekly_dates, open("weekly_marketcap.p", "wb"))
        weekly_dates_backup = pickled_weekly_dates
        time_to_sleep = random.randint(10,15)
        print "\tSleeping "+str(time_to_sleep)+" seconds"
        time.sleep(time_to_sleep)
        

In [76]:
# updates pickled weekly dates by crawling for updated market cap values
# returns weekly dates as a list of tuples

def update_pickled_weekly_dates():
    try:
        pickled_weekly_dates = pickle.load( open("weekly_marketcap.p", "rb"))
        weekly_dates = get_weekly_dates()
        weeks_behind = len(weekly_dates) - len(pickled_weekly_dates)
        if (weeks_behind > 0):
            # behind
            print "Spider bootup..."
            weeks_to_crawl = weekly_dates[-weeks_behind:]
            print str(len(weeks_to_crawl))+" behind..."
            market_cap_crawler(weeks_to_crawl, pickled_weekly_dates)
            print "Spider shutdown..."
        return pickled_weekly_dates
            
    except (OSError, IOError) as e:
        # create new pickle file if it doesn't exist
        pickle.dump([], open("weekly_marketcap.p", "wb"))
        
update_pickled_weekly_dates()


[('20130428', u'1,583,440,000'),
 ('20130505', u'1,370,880,000'),
 ('20130512', u'1,355,529,984'),
 ('20130519', u'1,445,040,000'),
 ('20130526', u'1,573,330,048'),
 ('20130602', u'1,437,159,936'),
 ('20130609', u'1,191,800,064'),
 ('20130616', u'1,176,669,952'),
 ('20130623', u'1,291,449,984'),
 ('20130630', u'1,172,969,984'),
 ('20130707', u'931,886,976'),
 ('20130714', u'1,139,960,064'),
 ('20130721', u'1,116,540,032'),
 ('20130728', u'1,219,320,064'),
 ('20130804', u'1,337,120,000'),
 ('20130811', u'1,316,160,000'),
 ('20130818', u'1,432,429,952'),
 ('20130825', u'1,542,210,048'),
 ('20130901', u'1,725,049,984'),
 ('20130908', u'1,541,709,952'),
 ('20130915', u'1,648,640,000'),
 ('20130922', u'1,671,500,032'),
 ('20130929', u'1,782,739,968'),
 ('20131006', u'1,664,640,000'),
 ('20131013', u'1,784,710,016'),
 ('20131020', u'2,192,689,920'),
 ('20131027', u'2,459,320,064'),
 ('20131103', u'2,707,279,872'),
 ('20131110', u'4,063,399,936'),
 ('20131117', u'6,172,020,224'),
 ('20131124'

In [43]:
def convert_date_to_datecode(date):
    month = "0"+str(date.month) if date.month < 10 else date.month
    day = "0"+str(date.day) if date.day < 10 else date.day
    datecode = str(date.year)+str(month)+str(day)
    assert(len(datecode) == 8)
    return datecode

In [73]:
# Calculate date objects (every 7 days)
# returns list of dates

def get_weekly_dates():
    year = int(start_date_code[0:4])
    month = int(start_date_code[4:6])
    day = int(start_date_code[6:8])
    
    start_date = datetime.date(year, month, day)
    today = datetime.date.today()
    days_diff_datetime = today - start_date
    days_diff = days_diff_datetime.days+7

    return [convert_date_to_datecode(start_date + datetime.timedelta(days=i)) for i in range(0, days_diff, 7)]

In [45]:
# Pulls the market cap for that week
# date_code {string} - datecode for that week in format YYYYMMDD
def get_total_market_cap(date_code, retries=0):
    url = "https://coinmarketcap.com/historical/"+date_code
    print "\turl: "+url
    r = requests.get(url)
    print "\tstatus code:"+str(r.status_code)
    html = r.text
    total_market_cap = re.findall('Total Market Cap: \$([\d,]+)<\/strong', html)
    try:
        print "\tMarket cap found! - "+total_market_cap[0]
        return total_market_cap[0]
    except IndexError:
        print "\tCouldn't get market cap"
        if (r.status_code >= 400):
            retries+=1
            sleeping = retries*random.randint(20,40)
            print "\tSleeping "+str(sleeping)+" seconds"
            time.sleep(sleeping)
            requests.get("https://coinmarketcap.com/currencies/bitcoin/")
            time.sleep(5)
            if (r.status_code == 404 and retries == 3):
                return None
            return get_total_market_cap(date_code, retries)
        return None


In [66]:
def get_coin_historical_data(coin_path, coin_slug, cookie):
    historical_url = get_historical_url(coin_path)
    print "Url:"+historical_url
    headers = {
        'authority': 'www.coingecko.com',
        'method': 'GET',
        'path':'/en/coins/bitcoin/historical_data/usd?end_date=2020-07-10&start_date=2008-07-03',
        'scheme': 'https',
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'accept-encoding': 'gzip, deflate',
        'accept-language': 'en-US,en;q=0.9',
        'cache-control': 'max-age=0',
        'cookie': cookie,
        'dnt':'1',
        'sec-fetch-dest':'document',
        'sec-fetch-mode':'navigate',
        'sec-fetch-site':'same-origin',
        'sec-fetch-user':'?1',
        'upgrade-insecure-requests':'1',
        'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'
    }
    r = requests.get(historical_url, headers=headers, stream=True)
    html = r.text
    history = re.findall(r'class="font-semibold text-center">(\d{4}-\d{2}-\d{2})<\/th>\s<td class="text-center">\s\$[\d,]+.\d{2}\s<\/td>\s<td class="text-center">\s\$([\d,]+.\d{2})\s<\/td>\s<\/tr>', html)
    return history[0::7] # get every 7th element


In [67]:
def get_historical_url(coin_path):
    current_date = datetime.date.today().strftime('%Y-%m-%d')
    return "https://www.coingecko.com/en/coins/"+coin_path+"/historical_data/usd?end_date="+current_date+"&start_date=2008-07-03#panel"

In [68]:
def pull_coin_list():
    r = requests.get("https://www.coingecko.com/en");
    html = r.text
    coin_list = re.findall(r'font-bold\"\shref="\/en\/coins\/(.*)\">\s(.*)\s<', html)
    assert(len(coin_list) == 100)
    return coin_list

In [74]:
# MAIN
coin_list = pull_coin_list()
# print get_coin_historical_data('bitcoin','BTC', GECKO_COOKIE)
print get_weekly_dates()[-1:]

['20200712']
